In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
path = "../data/Formula 1 World Championship (1950 - 2023) Kaggle"

# Load data
df_circuits = pd.read_csv(path + "/circuits.csv")
df_constructor_results = pd.read_csv(path + "/constructor_results.csv")
df_constructor_standings = pd.read_csv(path + "/constructor_standings.csv")
df_constructors = pd.read_csv(path + "/constructors.csv")
df_driver_standings = pd.read_csv(path + "/driver_standings.csv")
df_drivers = pd.read_csv(path + "/drivers.csv")
df_lap_times = pd.read_csv(path + "/lap_times.csv")
df_pit_stops = pd.read_csv(path + "/pit_stops.csv")
df_qualifying = pd.read_csv(path + "/qualifying.csv")
df_races = pd.read_csv(path + "/races.csv")
df_results = pd.read_csv(path + "/results.csv")
df_seasons = pd.read_csv(path + "/seasons.csv")
df_sprint_results = pd.read_csv(path + "/sprint_results.csv")
df_status = pd.read_csv(path + "/status.csv")